### Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings ; warnings.filterwarnings('ignore')

### 주택수, 단독주택, 다가구주택수
1인 가구의 영향력을 보기 위해 주택수를 구하며 1인 가구의 주거주형태인 단독주택, 다가구주택수를 구한다.
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
한 번에 6개년 데이터를 다운받으려 하면 오류가 나 3개년씩 나누어 저장한다.
- [주택분포] https://data.seoul.go.kr/dataList/10585/S/2/datasetView.do

In [2]:
housetype17 = pd.read_csv('RawData/주택종류별+주택-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20230417222822.csv').iloc[4:]
housetype20 = pd.read_csv('RawData/주택종류별+주택-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20230417222808.csv').iloc[4:]

In [3]:
housetype = pd.DataFrame()

for i in range(3,30,9):
    ease = pd.concat([housetype17.iloc[:, 1], housetype17.iloc[:,[i,i+1,i+3,i+5]]], axis=1)
    ease.columns = ['자치구','주택수','단독주택수', '다가구주택수','아파트수']
    ease['년도'] = int(housetype17.columns[i])
    housetype = pd.concat([housetype, ease])
    
for i in range(3,30,9):
    ease = pd.concat([housetype20.iloc[:, 1], housetype20.iloc[:,[i,i+1,i+3,i+5]]], axis=1)
    ease.columns = ['자치구','주택수','단독주택수', '다가구주택수','아파트수']
    ease['년도'] = int(housetype20.columns[i])
    housetype = pd.concat([housetype, ease])
    
housetype[['주택수', '단독주택수', '다가구주택수', '아파트수']] = housetype[['주택수', '단독주택수', '다가구주택수', '아파트수']].astype('i')
housetype.reset_index(drop=True, inplace=True)
print(housetype.shape)

(150, 6)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- [주택수] https://data.seoul.go.kr/dataList/10585/S/2/datasetView.do

In [4]:
GJ_housetype = pd.read_csv('RawData/주택종류별+주택-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20230423115920.csv')
GJ_housetype = GJ_housetype[GJ_housetype.iloc[:,1]=="광진구"].iloc[1:,[2,3,4,6,8]].reset_index(drop=True)
GJ_housetype.columns = ['행정동','주택수','단독주택수', '다가구주택수','아파트수']
GJ_housetype[['주택수', '단독주택수', '다가구주택수', '아파트수']] = GJ_housetype[['주택수', '단독주택수', '다가구주택수', '아파트수']].astype('i')
GJ_housetype.shape

(15, 5)

### 전월세 거래와 다가구주택의 거래 정보
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [전월세가격] https://data.seoul.go.kr/dataList/OA-21276/S/1/datasetView.do

In [5]:
rent = pd.DataFrame()

for i in range(2015, 2021):
    raw_rent = pd.read_table(f'RawData/서울열린광장_전월세가/서울특별시_전월세가_{i}.txt', sep=',',encoding='cp949',
                             usecols=[2,11,12,13,14,17])
    
    # 아파트를 월세로 거래하는 사람은 극히 일부기에 월세와 관해선 다가구주택 정보만 생성한다.
    monthly = raw_rent.query('전월세구분=="월세"')
    ease_m = pd.concat([monthly.groupby('자치구명')['임대면적'].size(),
                        monthly.groupby('자치구명')[['임대면적','보증금(만원)','임대료(만원)']].mean()], axis=1)

    monthly = monthly.query('건물용도=="단독다가구"')
    ease_m = pd.concat([ease_m,
                        monthly.groupby('자치구명')['임대면적'].size(),
                        monthly.groupby('자치구명')[['임대면적','보증금(만원)','임대료(만원)']].mean()], axis=1)

    ease_m.columns = ['월세거래수','월세평균임대면적','월세평균보증금액','월세평균임대료액']\
                      +['다가구주택_월세거래수', '다가구주택_월세평균임대면적','다가구주택_월세평균보증금액','다가구주택_월세평균임대료액']

    # 다가구주택과 아파트의 전세 정보를 생성하며 전세는 임대료를 내지 않기에 임대료 정보는 생성하지 않는다.
    charter = raw_rent.query('전월세구분=="전세"')
    ease_c = pd.concat([charter.groupby('자치구명')['임대면적'].size(),
                        charter.groupby('자치구명')[['임대면적','보증금(만원)']].mean()], axis=1)

    charter = charter.query('건물용도=="단독다가구" | 건물용도=="아파트"')
    ease_c = pd.concat([ease_c,
                        pd.pivot_table(charter, index='자치구명', columns='건물용도', values='임대면적', aggfunc=np.size),
                        pd.pivot_table(charter, index='자치구명', columns='건물용도', 
                                       values=['임대면적','보증금(만원)'], aggfunc=np.mean)], axis=1)

    ease_c.columns = ['전세거래수','전세평균임대면적','전세평균보증금액']\
                      +['다가구주택_전세거래수', '아파트_전세거래수']\
                      +['다가구주택_전세평균임대면적', '아파트_전세평균임대면적',
                        '다가구주택_전세평균보증금액', '아파트_전세평균보증금액']
    ease = pd.concat([ease_m, ease_c], axis=1).reset_index().rename(columns={'자치구명':'자치구'})
    ease['임대거래수'] = ease['월세거래수']+ease['전세거래수']
    ease['년도'] = i
    rent = pd.concat([rent, ease])
    
rent.reset_index(drop=True, inplace=True)
print(rent.shape)

(150, 20)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리

In [6]:
raw_rent = pd.read_table('RawData/서울열린광장_전월세가/서울특별시_전월세가_2020.txt', sep=',',encoding='cp949',
                         usecols=[2,4,11,12,13,14,17]).query('자치구명=="광진구"')

In [7]:
# 아파트를 월세로 거래하는 사람은 극히 일부기에 월세와 관해선 다가구주택 정보만 생성한다.
monthly = raw_rent.query('전월세구분=="월세"')
ease_m = pd.concat([monthly.groupby('법정동명')['임대면적'].size(),
                    monthly.groupby('법정동명')[['임대면적','보증금(만원)','임대료(만원)']].mean()], axis=1)

monthly = monthly.query('건물용도=="단독다가구"')
ease_m = pd.concat([ease_m,
                    monthly.groupby('법정동명')['임대면적'].size(),
                    monthly.groupby('법정동명')[['임대면적','보증금(만원)','임대료(만원)']].mean()], axis=1)

ease_m.columns = ['월세거래수','월세평균임대면적','월세평균보증금액','월세평균임대료액']\
                  +['다가구주택_월세거래수', '다가구주택_월세평균임대면적','다가구주택_월세평균보증금액','다가구주택_월세평균임대료액']

In [8]:
# 다가구주택과 아파트의 전세 정보를 생성하며 전세는 임대료를 내지 않기에 임대료 정보는 생성하지 않는다.
charter = raw_rent.query('전월세구분=="전세"')
ease_c = pd.concat([charter.groupby('법정동명')['임대면적'].size(),
                    charter.groupby('법정동명')[['임대면적','보증금(만원)']].mean()], axis=1)

charter = charter.query('건물용도=="단독다가구" | 건물용도=="아파트"')
ease_c = pd.concat([ease_c,
                    pd.pivot_table(charter, index='법정동명', columns='건물용도', values='임대면적', aggfunc=np.size),
                    pd.pivot_table(charter, index='법정동명', columns='건물용도', 
                                   values=['임대면적','보증금(만원)'], aggfunc=np.mean)], axis=1)

ease_c.columns = ['전세거래수','전세평균임대면적','전세평균보증금액']\
                  +['다가구주택_전세거래수', '아파트_전세거래수']\
                  +['다가구주택_전세평균임대면적', '아파트_전세평균임대면적',
                    '다가구주택_전세평균보증금액', '아파트_전세평균보증금액']

In [9]:
GJ_rent = pd.concat([ease_m, ease_c], axis=1)
GJ_rent['임대거래수'] = GJ_rent['월세거래수']+GJ_rent['전세거래수']

# 법정동으로 작성되어 있어 행정동으로 개수를 늘려야 한다.
GJ_rent = pd.concat([GJ_rent, 
                     pd.DataFrame([GJ_rent.loc["중곡동"].tolist()]*4+[GJ_rent.loc["자양동"].tolist()]*4+[GJ_rent.loc["구의동"].tolist()]*3, 
                                  index=[f'중곡{i}동' for i in range(1,5)]+[f'자양{i}동' for i in range(1,5)]+[f'구의{i}동' for i in range(1,4)],
                                   columns=GJ_rent.columns)])
GJ_rent.drop(['중곡동','자양동','구의동'], inplace=True)
GJ_rent.reset_index(inplace=True)
GJ_rent = GJ_rent.rename(columns={'index':'행정동'})
print(GJ_rent.shape)

(15, 19)


### 부동산 실거래가
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [실거래가] https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do

In [10]:
raw_rp = pd.read_csv('RawData/서울시 부동산 실거래가 정보.csv', encoding='cp949',
                     usecols=[0,2,4,11,12,18]).query('2015 <= 접수연도 <= 2020')

In [11]:
rp = raw_rp.groupby(['자치구명','접수연도']).agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})

raw_rp2 = raw_rp.query('건물용도=="단독다가구"')
rp = pd.concat([rp,
                raw_rp2.groupby(['자치구명','접수연도']).agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})], axis=1)

raw_rp2 = raw_rp.query('건물용도=="아파트"')
rp = pd.concat([rp,
                raw_rp2.groupby(['자치구명','접수연도']).agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})], axis=1).reset_index()

rp.columns = ['자치구','년도','실거래수','평균실거래가','평균거래면적']\
              +['다가구주택_실거래수','다가구주택_평균실거래가','다가구주택_평균거래면적']\
              +['아파트_실거래수','아파트_평균실거래가','아파트_평균거래면적']
print(rp.shape)

(150, 11)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- 서울 자치구별 데이터를 재사용한다.

In [12]:
raw_rp = raw_rp.query('자치구명=="광진구" & 접수연도== 2020')

In [13]:
GJ_rp = raw_rp.groupby('법정동명').agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})

raw_rp2 = raw_rp.query('건물용도=="단독다가구"')
GJ_rp = pd.concat([GJ_rp,
                   raw_rp2.groupby('법정동명').agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})], axis=1)

raw_rp2 = raw_rp.query('건물용도=="아파트"')
GJ_rp = pd.concat([GJ_rp,
                   raw_rp2.groupby('법정동명').agg({'물건금액(만원)':[np.size, np.mean], '건물면적(㎡)':np.mean})], axis=1)

GJ_rp.columns = ['실거래수','평균실거래가','평균거래면적']\
                +['다가구주택_실거래수','다가구주택_평균실거래가','다가구주택_평균거래면적']\
                +['아파트_실거래수','아파트_평균실거래가','아파트_평균거래면적']

In [14]:
# 법정동으로 작성되어 있어 행정동으로 개수를 늘려야 한다.
GJ_rp = pd.concat([GJ_rp, 
                  pd.DataFrame([GJ_rp.loc["중곡동"].tolist()]*4+[GJ_rp.loc["자양동"].tolist()]*4+[GJ_rp.loc["구의동"].tolist()]*3, 
                                index=[f'중곡{i}동' for i in range(1,5)]+[f'자양{i}동' for i in range(1,5)]+[f'구의{i}동' for i in range(1,4)],
                                columns=GJ_rp.columns)])
GJ_rp.drop(['중곡동','자양동','구의동'], inplace=True)
GJ_rp.reset_index(inplace=True)
GJ_rp = GJ_rp.rename(columns={'index':'행정동'})
print(GJ_rp.shape)

(15, 10)


### 지가변동률
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [통계청] https://kosis.kr/statHtml/statHtml.do?orgId=408&tblId=DT_31501N_010&vw_cd=MT_ZTITLE&list_id=I2_7&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=K1&path=%25EA%25B5%25AD%25ED%2586%25A0%25EC%259D%25B4%25EC%259A%25A9%2520%253E%2520%25EC%25A0%2584%25EA%25B5%25AD%25EC%25A7%2580%25EA%25B0%2580%25EB%25B3%2580%25EB%258F%2599%25EB%25A5%25A0%25EC%25A1%25B0%25EC%2582%25AC%25EC%259D%258D%25EB%25A9%25B4%25EB%258F%2599%25EB%25B3%2584%2520%25EC%25A7%2580%25EA%25B0%2580%25EB%25B3%2580%25EB%258F%2599%25EB%25A5%25A0

In [15]:
raw_fr_land = pd.DataFrame()

for i in os.listdir('RawData/통계청_지가변동률'):
    ease = pd.read_csv(f'RawData/통계청_지가변동률/{i}', encoding='cp949')
    ease = ease[(ease.iloc[:,0]=="서울") & (ease.iloc[:,2]=="소계")].iloc[1:].set_index('읍면동별(2)').iloc[:,2:]
    raw_fr_land = pd.concat([raw_fr_land, ease], axis=1)
    
raw_fr_land = raw_fr_land.astype('f')

In [16]:
fr_land = pd.DataFrame()

for i in range(2016,2021):
    ease = raw_fr_land[[x for x in raw_fr_land.columns if str(i) in x]].mean(axis=1).reset_index()
    ease.columns = ['자치구','지가변동률']
    ease['년도'] = i
    fr_land = pd.concat([fr_land, ease])
    
fr_land.reset_index(drop=True, inplace=True)
print(fr_land.shape)

(125, 3)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- 서울 자치구별 데이터를 재사용한다.

In [17]:
raw_fr_land = pd.DataFrame()

for i in os.listdir('RawData/통계청_지가변동률'):
    ease = pd.read_csv(f'RawData/통계청_지가변동률/{i}', encoding='cp949')
    for j in ease.columns:
        if '2020' in j:
            ease = ease[ease.iloc[:,1]=="광진구"].iloc[1:].set_index('읍면동별(3)').iloc[:,2:]
            raw_fr_land = pd.concat([raw_fr_land, ease], axis=1)
            break
    
raw_fr_land = raw_fr_land.astype('f')

In [18]:
GJ_fr_land = raw_fr_land.iloc[:,-12:].mean(axis=1)
GJ_fr_land = pd.concat([GJ_fr_land, 
                        pd.Series([GJ_fr_land.loc['중곡동']]*4+[GJ_fr_land.loc['자양동']]*4+[GJ_fr_land.loc['구의동']]*3,
                                  index=[f'중곡{i}동' for i in range(1,5)]+[f'자양{i}동' for i in range(1,5)]+[f'구의{i}동' for i in range(1,4)],
                                  name='읍면동별(3)')])

GJ_fr_land.drop(['중곡동','자양동','구의동'], inplace=True)
GJ_fr_land = GJ_fr_land.reset_index()
GJ_fr_land.columns = ['행정동','지가변동률']
print(GJ_fr_land.shape)

(15, 2)


### Merge Datas

In [19]:
seoul = housetype.merge(rent, on=['자치구','년도']).merge(rp, on=['자치구','년도']).merge(fr_land, on=['자치구','년도'])
print(seoul.shape)

(125, 34)


In [20]:
gwangjin = GJ_housetype.merge(GJ_rent, on='행정동').merge(GJ_rp, on='행정동').merge(GJ_fr_land, on='행정동')
print(gwangjin.shape)

(15, 33)


## Feature Generation

In [21]:
seoul['아파트비율'] = seoul['아파트수'].divide(seoul['주택수'], axis=0)
gwangjin['아파트비율'] = gwangjin['아파트수'].divide(gwangjin['주택수'], axis=0)

In [22]:
seoul['단독주택비율'] = seoul['단독주택수'].divide(seoul['주택수'], axis=0)
gwangjin['단독주택비율'] = gwangjin['단독주택수'].divide(gwangjin['주택수'], axis=0)

In [23]:
seoul['단독주택_내_다가구주택_비율'] = seoul['다가구주택수'].divide(seoul['단독주택수'], axis=0)
gwangjin['단독주택_내_다가구주택_비율'] = gwangjin['다가구주택수'].divide(gwangjin['단독주택수'], axis=0)

In [24]:
seoul['월세비율'] = seoul['월세거래수'].divide(seoul['임대거래수'], axis=0)
gwangjin['월세비율'] = gwangjin['월세거래수'].divide(gwangjin['임대거래수'], axis=0)

In [25]:
seoul['다가구주택_전월세비율'] = seoul['다가구주택_월세거래수'].divide(seoul['다가구주택_전세거래수'], axis=0)
gwangjin['다가구주택_전월세비율'] = gwangjin['다가구주택_월세거래수'].divide(gwangjin['다가구주택_전세거래수'], axis=0)

In [26]:
seoul['다가구주택_실거래비율'] = seoul['다가구주택_실거래수'].divide(seoul['실거래수'], axis=0)
gwangjin['다가구주택_실거래비율'] = gwangjin['다가구주택_실거래수'].divide(gwangjin['실거래수'], axis=0)

In [27]:
seoul['아파트_실거래비율'] = seoul['아파트_실거래수'].divide(seoul['실거래수'], axis=0)
gwangjin['아파트_실거래비율'] = gwangjin['아파트_실거래수'].divide(gwangjin['실거래수'], axis=0)

## Save Data

In [28]:
# 각 데이터를 저장한다.
seoul.to_csv('ProcessedData/주거정보_seoul.csv', index=False)
gwangjin.to_csv('ProcessedData/주거정보_gwangjin.csv', index=False)